# Module 2 Final Project
Emily J. Cain

## Hypothesis 1
* H0 - Discount amount does not have a statistically significant effect on the quantity of the product ordered.
* H1 - Discount amount has a statistically significant effect on the quantity of the product ordered.

### Further Questions
* At what level(s) of discount?

## Hypothesis 2
* H0 -
* H2 -

## Hypothesis 3
* H0 -
* H3 -

## Hypothesis 4
* H0 -
* H4 - 

## Ideas
* Does the database show how long a customer has been with the company? Does customer loyalty have an effect on orders? Actionable - target specific customers
* 

## Establish connection to the database

In [1]:
import sqlite3

In [2]:
connection = sqlite3.connect('Northwind_small.sqlite')
c = connection.cursor()

In [4]:
c.execute("SELECT name FROM sqlite_master WHERE type = 'table'").fetchall()

[('Employee',),
 ('Category',),
 ('Customer',),
 ('Shipper',),
 ('Supplier',),
 ('Order',),
 ('Product',),
 ('OrderDetail',),
 ('CustomerCustomerDemo',),
 ('CustomerDemographic',),
 ('Region',),
 ('Territory',),
 ('EmployeeTerritory',)]